# Col de Porte Observations

In [1]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from scipy.stats.stats import pearsonr

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

# Offline Turbulence Package
import turbpy

# Customize
sns.set_style("whitegrid")
sns.set_context('paper')
%matplotlib inline

In [2]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dirPre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dirPre = '/Users/karllapo/gdrive/'

dirProj = dirPre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + 'Graphics'
dirData = dirProj + 'data'

# Original Snoqualmie observations
dirDataRaw = dirProj + 'data/CDP'

# Sub-functions for calculating dewpoint/RH

In [3]:
def calc_RH(p,q,T):
    RH = np.empty_like(T)
    if np.nanmin(T) < 200:
        T = T + 273.16
    T_0 = 273.16

    RH = .263*p*q* (np.exp( (17.67*(T-T_0)) / (T-29.65) ))**(-1)
    RH[RH > 100] = 100
    return(RH)

def calc_Tdew(T,RH):
    # Unit checks
    if np.nanmax(T) > 100 or np.nanmin(T) > 40:
        raise ValueError('Air temperature must be in Celsius')

    if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
        RH = RH/100
        if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
            raise ValueError('Relative humidity must be a fraction on [0,1]')

    if not np.size(RH) == np.size(T):
        raise ValueError('Relative humidity and air temperature must have the same number of elements')

    # -----------------------------------------------------------------------------------------------
    # ALGORITHM
    # When are we calculating with respect to frost or water?
    frost_ind = np.flatnonzero(T <= 0)
    water_ind = np.flatnonzero(T > 0)
    # Frost coefficients 
    b_frost = 22.587
    c_frost = 273.86
    # Water coefficients
    b_water = 17.625
    c_water = 243.03

    # Pre-allocate
    Tdew = np.empty_like(T)

    Tdew[frost_ind] = MagnusTetens(T[frost_ind],RH[frost_ind],b_frost,c_frost)
    Tdew[water_ind] = MagnusTetens(T[water_ind],RH[water_ind],b_water,c_water)

    return(Tdew)

# SUB-FUNCTION for actual expression
def MagnusTetens(T,RH,b,c):
    dew = (c*( np.log(RH) + (b * T)/(c + T) )) / ( b - np.log(RH) - (b * T)/(c + T) )
    return(dew)

## CDP Model Forcing Data

In [4]:
os.chdir(dirDataRaw)
CDP = xr.open_dataset('CDP_met_insitu.nc')
dropVarList = ['aspect', 'slope', 'ZREF', 'UREF', 'FORC_TIME_STEP', 'Wind_DIR',
               'CO2air', 'NEB', 'Tair_flag', 'Qair_flag', 'Wind_flag', 'Rainf_flag',
               'Snowf_flag', 'LWdown_flag', 'DIR_SWdown_flag', 'SCA_SWdown_flag', 
               'NEB_flag', 'HUMREL_flag']
CDP = CDP.drop(dropVarList)
CDP['SWdwn'] = CDP['DIR_SWdown'] + CDP['SCA_SWdown']
CDP = CDP.rename({'PSurf': 'Press', 'Qair': 'QS', 'LWdown': 'LWdwn',
                  'Rainf': 'precipRain', 'Snowf': 'precipSnow', 'Wind': 'WIND'})
CDP = CDP.drop(['DIR_SWdown', 'SCA_SWdown'])
CDP = CDP.squeeze('location')
CDP['Tair'] = CDP.Tair - 273.15
CDP['precip'] = CDP.precipRain + CDP.precipSnow
print(CDP)


<xarray.Dataset>
Dimensions:     (time: 157776)
Coordinates:
  * time        (time) datetime64[ns] 1993-08-01 1993-08-01T00:59:59.999987 ...
Data variables:
    lat         float64 45.3
    lon         float64 5.77
    altitude    float64 1.325e+03
    Tair        (time) float64 9.288 9.205 9.122 9.048 8.978 8.541 8.603 ...
    QS          (time) float64 0.006398 0.006309 0.006256 0.006203 0.00615 ...
    WIND        (time) float64 1.433 1.233 1.034 0.835 0.736 0.501 0.504 ...
    precipRain  (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    precipSnow  (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    LWdwn       (time) float64 326.7 324.2 321.2 280.0 289.6 290.8 292.5 ...
    Press       (time) float64 8.766e+04 8.76e+04 8.759e+04 8.754e+04 ...
    HUMREL      (time) float64 75.67 75.04 74.82 74.57 74.28 75.85 74.88 ...
    SWdwn       (time) float64 0.0 0.0 0.0 0.0 0.0 86.44 223.7 371.6 537.8 ...
    precip      (time) float64 0.0 0.0 0.0 0.0 0.0 0.

## Read supporting evaluation data

In [5]:
# date parsing function for pandas csv_read
def parse(timeStr):
    s = '00'
    date_str = timeStr + ':' + s
    dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S")
    return dt

# -------------------------------------------------------------------------
# Read supporting data
# 19 rows of comments
os.chdir(dirDataRaw)
datafile = 'CDP_hor_eval.tab'
support = pd.read_csv(datafile,
                  sep='\t',
                  header=0,
                  skiprows=20,
                  date_parser=parse,
                  parse_dates={'time': [0]},
                  index_col='time'
                 )

# Use my naming convention
support.columns = ['snowDepth', 'Tsrf', 'Lysimeter1', 'Lysimeter2',
               'groundHeatFlux1', 'groundHeatFlux2', 'groundHeatFlux3', 'Albedo']
support.drop(['Lysimeter1', 'Lysimeter2', 'groundHeatFlux1', 'groundHeatFlux2', 'groundHeatFlux3'],
             axis=1, inplace=True)

# -------------------------------------------------------------------------
# Convert to xarray Dataset
support = xr.Dataset(support)

# Take daily average, reindex to half-hourly time series, use in snow presence criteria
TsrfDaily = support.Tsrf.resample(how='mean', freq='d', dim='time', label='left')
TsrfDaily = TsrfDaily.reindex_like(support, method='ffill')

# bare ground when no snowdepth recorded or the daily surface temperature is above freezing
groundSurfTemp = support.Tsrf[(support.snowDepth == 0) | (TsrfDaily > 0.5)]

# snow covered ground when snow is observed and the surface temperature is below freezing
snowSurfTemp = support.Tsrf[(support.snowDepth > 0) & (TsrfDaily < 0.5)]

# Assign to support xarray.Dataset
support['groundTs'] = groundSurfTemp
support['snowTs'] = snowSurfTemp

# Create snow presence variable
snowPres = ((support.snowDepth > 0) | (TsrfDaily < 0.5))
support['SP'] = snowPres.astype(int)
support = support.reindex_like(CDP.time, method='nearest')


//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1143: RuntimeWarning: invalid value encountered in greater
  if not reflexive
//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1143: RuntimeWarning: invalid value encountered in less
  if not reflexive


In [6]:
# -------------------------------------------------------------------------
# Add necessary support data
# Many other variables are available, but I'm selecting only a small collection of them
CDP['Albedo'] = support['Albedo']
CDP['Tsrf'] = support['Tsrf']
CDP['SP'] = support['SP']
CDP['snowTs'] = support['snowTs']
CDP['snowDepth'] = support['snowDepth']
CDP['groundTs'] = support['groundTs']

# Bulk Richardson
CDP['RiBulk'],_,_ = turbpy.bulkRichardson(CDP.Tair + 273.15, CDP.Tsrf + 273.15, CDP.WIND, 1.5 - CDP.snowDepth)

# Again, there is a weird encoding issue with the date
CDP.coords['time'] = pd.date_range(CDP.time.values[0], datetime(2011, 7, 31, 23, 0, 0), freq='H')

# -------------------------------------------------------------------------
# Write netcdf
os.chdir(dirData)
CDP.to_netcdf('CDP.ModTsfc.nc')
print(CDP)

//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1143: RuntimeWarning: divide by zero encountered in true_divide
  if not reflexive
//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1143: RuntimeWarning: invalid value encountered in true_divide
  if not reflexive


<xarray.Dataset>
Dimensions:     (time: 157776)
Coordinates:
  * time        (time) datetime64[ns] 1993-08-01 1993-08-01T01:00:00 ...
Data variables:
    lat         float64 45.3
    lon         float64 5.77
    altitude    float64 1.325e+03
    Tair        (time) float64 9.288 9.205 9.122 9.048 8.978 8.541 8.603 ...
    QS          (time) float64 0.006398 0.006309 0.006256 0.006203 0.00615 ...
    WIND        (time) float64 1.433 1.233 1.034 0.835 0.736 0.501 0.504 ...
    precipRain  (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    precipSnow  (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    LWdwn       (time) float64 326.7 324.2 321.2 280.0 289.6 290.8 292.5 ...
    Press       (time) float64 8.766e+04 8.76e+04 8.759e+04 8.754e+04 ...
    HUMREL      (time) float64 75.67 75.04 74.82 74.57 74.28 75.85 74.88 ...
    SWdwn       (time) float64 0.0 0.0 0.0 0.0 0.0 86.44 223.7 371.6 537.8 ...
    precip      (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0